# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [10]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [11]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=100)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=100)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
print(len(cities))

78


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [12]:
#Creating lists to hold values to later populate dataframe
lats = []
lons = []
max_temps = []
humidities = []
clouds = []
wind_speeds = []
countries =  []
dates = []
skip_counter = 0

#Beginning of print process
print("Beginning Data Retrieval")
print("------------------------------")

#Loop through each city in cities and generate json link to pull data
for name in cities:
    weather_url = f"https://api.openweathermap.org/data/2.5/weather?q={name}&appid={weather_api_key}&units=metric"
    response = requests.get(weather_url).json()

    #Catch exception in case no data found for selected city
    try:
        #Store data into variables
        lat = response["coord"]["lat"]
        lon = response["coord"]["lon"]
        max_temp = response["main"]["temp_max"]
        humidity = response["main"]["humidity"]
        cloudiness = response["clouds"]["all"]
        wind_speed = response["wind"]["speed"]
        country = response["sys"]["country"]
        date = response["dt"]

        #Append data to corresponding lists
        lats.append(lat)
        lons.append(lon)
        max_temps.append(max_temp)
        humidities.append(humidity)
        clouds.append(cloudiness)
        wind_speeds.append(wind_speed)
        countries.append(country)
        dates.append(date)

        #Print log of city being processed
        print(f"Processing record {cities.index(name)} | {name.capitalize()}")
        print(f"Data visual: {len(lats)}\n")
    except KeyError:
        print(f"No data found for {name.capitalize()}. Skipping...")

        #Remove city from cities list for easier dataframe creation later
        #cities.remove(name)
        lats.append("")
        lons.append("")
        max_temps.append("")
        humidities.append("")
        clouds.append("")
        wind_speeds.append("")
        countries.append("")
        dates.append("")
        print(f"Data visual: {len(lats)}\n")
        skip_counter += 1

#End of print process
print("-----------------------------")
print("Data Retrieval Complete")
print("-----------------------------")

print(f"Number of Skips: {skip_counter}")

Beginning Data Retrieval
------------------------------
No data found for Vaitupu. Skipping...
Data visual: 1

Processing record 1 | Bluff
Data visual: 2

Processing record 2 | Castro
Data visual: 3

Processing record 3 | Borovskoy
Data visual: 4

Processing record 4 | Pacifica
Data visual: 5

Processing record 5 | Hithadhoo
Data visual: 6

No data found for Illoqqortoormiut. Skipping...
Data visual: 7

Processing record 7 | Vaini
Data visual: 8

Processing record 8 | Port elizabeth
Data visual: 9

Processing record 9 | Muisne
Data visual: 10

Processing record 10 | Jamestown
Data visual: 11

Processing record 11 | Clyde river
Data visual: 12

Processing record 12 | Sabha
Data visual: 13

Processing record 13 | Albany
Data visual: 14

Processing record 14 | Saryagash
Data visual: 15

Processing record 15 | Saldanha
Data visual: 16

Processing record 16 | General jose eduvigis diaz
Data visual: 17

Processing record 17 | Busselton
Data visual: 18

Processing record 18 | Bathsheba
Data v

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [13]:
#Create dataframe
raw_data = {"City": cities, "Lat": lats, "Lon": lons, "Max Temp": max_temps, "Humidity": humidities, "Cloudiness": clouds, "Wind Speed": wind_speeds, "Country": countries, "Date": dates}

countries_df = pd.DataFrame(raw_data)

#Find indexes where Lat is blank for future removal
index_names = countries_df[countries_df["Lat"] == ""].index

countries_df.drop(index_names, inplace = True)

#Export dataframe to a csv file
countries_df.to_csv("../output_data/cities.csv", index = False, header = True)

#Print out cleaned dataframe
countries_df









,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,bluff,-46.6,168.33,2.78,90,87,0.89,NZ,1602685989
2,castro,-24.79,-50.01,29.39,37,28,2.37,BR,1602685989
3,borovskoy,53.8,64.15,10.84,45,25,4.86,KZ,1602685990
4,pacifica,37.61,-122.49,17.78,72,20,1.5,US,1602685880
5,hithadhoo,-0.6,73.08,28.48,74,90,5.32,MV,1602685990
...,...,...,...,...,...,...,...,...,...
73,tuy hoa,13.08,109.3,25.42,88,100,1.63,VN,1602686010
74,geraldton,-28.77,114.6,17,77,40,3.6,AU,1602685852
75,saint-philippe,-21.36,55.77,26,74,20,3.6,RE,1602685460
76,raudeberg,61.99,5.14,8.89,72,8,9.72,NO,1602686010


In [15]:
over_100 = countries_df.loc[countries_df["Humidity"] < 90]

over_100

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,castro,-24.79,-50.01,29.39,37,28,2.37,BR,1602685989
3,borovskoy,53.8,64.15,10.84,45,25,4.86,KZ,1602685990
4,pacifica,37.61,-122.49,17.78,72,20,1.5,US,1602685880
5,hithadhoo,-0.6,73.08,28.48,74,90,5.32,MV,1602685990
7,vaini,-21.2,-175.2,21,83,40,3.1,TO,1602685459
...,...,...,...,...,...,...,...,...,...
73,tuy hoa,13.08,109.3,25.42,88,100,1.63,VN,1602686010
74,geraldton,-28.77,114.6,17,77,40,3.6,AU,1602685852
75,saint-philippe,-21.36,55.77,26,74,20,3.6,RE,1602685460
76,raudeberg,61.99,5.14,8.89,72,8,9.72,NO,1602686010


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.
humid_index = 

In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression